In [5]:
import os
import numpy as np
import scipy
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG19
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split


In [6]:
data_dir = "D:/Curated X-Ray Dataset/"
batch_size = 16
img_size = (224, 224)  # VGG16 input size

datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

validation_generator = datagen.flow_from_directory(
    data_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)


Found 7367 images belonging to 4 classes.
Found 1841 images belonging to 4 classes.


In [7]:
base_model = VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

for layer in base_model.layers:
    layer.trainable = False

In [8]:
# Flatten the output layer to 1 dimension
x = layers.Flatten()(base_model.output)

# Add a fully connected layer with softmax activation
output = layers.Dense(4, activation='softmax')(x)  # Assuming 4 classes

# Model without Added Layer
model = models.Model(base_model.input, output)

#Model with added layer
# model = models.Sequential()
# model.add(base_model)
# model.add(layers.Flatten())
# model.add(layers.Dense(256, activation='relu'))
# model.add(layers.Dropout(0.5))
# model.add(layers.Dense(4, activation='softmax'))  # Assuming 4 classes (Normal, Pneumonia-Viral, Pneumonia-Bacterial, COVID-19)


initial_learning_rate = 0.001
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate, decay_steps=100000, decay_rate=0.96, staircase=True
)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Compile without LR Schedule
# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the Model
epochs = 10  # You can adjust this based on your needs

In [9]:
history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator
)

# Evaluate the Model
_, accuracy = model.evaluate(validation_generator)
print(f"Validation Accuracy: {accuracy * 100:.2f}%")

Epoch 1/10
461/461 [==============================] - 94s 186ms/step - loss: 0.6278 - accuracy: 0.7711 - val_loss: 0.2806 - val_accuracy: 0.8968
Epoch 2/10
461/461 [==============================] - 73s 159ms/step - loss: 0.4834 - accuracy: 0.8258 - val_loss: 0.4578 - val_accuracy: 0.8533
Epoch 3/10
461/461 [==============================] - 74s 161ms/step - loss: 0.3723 - accuracy: 0.8535 - val_loss: 0.7666 - val_accuracy: 0.7192
Epoch 4/10
461/461 [==============================] - 77s 167ms/step - loss: 0.3299 - accuracy: 0.8689 - val_loss: 0.3915 - val_accuracy: 0.8322
Epoch 5/10
461/461 [==============================] - 76s 165ms/step - loss: 0.2812 - accuracy: 0.8898 - val_loss: 0.3693 - val_accuracy: 0.8696
Epoch 6/10
461/461 [==============================] - 77s 167ms/step - loss: 0.2952 - accuracy: 0.8868 - val_loss: 0.4241 - val_accuracy: 0.8653
Epoch 7/10
461/461 [==============================] - 76s 164ms/step - loss: 0.1918 - accuracy: 0.9211 - val_loss: 0.3039 - val_ac

In [10]:
import tensorflow as tf
tf.test.is_built_with_cuda()


True

In [11]:
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True